In [2]:
from numpy import *
import pandas as pd
import seaborn as sns
from math import log

In [3]:
def createDataSet():
    dataSet = [[1,1,'yes'],
              [1,1,'yes'],
              [1,0,'no'],
              [0,1,'no'],
              [0,1,'no']]
    labels = ['no surfing', 'flippers']
    return dataSet, labels

In [4]:
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts={}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel]=0
        labelCounts[currentLabel]+=1
    shannonEnt=0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries # numEntries : 5
        shannonEnt -=prob*log(prob,2)
    return shannonEnt

In [5]:
def splitDataSet(dataSet, axis, value):
    retDataSet=[]
    for featVec in dataSet:
        if featVec[axis]==value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [6]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0])-1 # 2개 ([1,1])
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/ float(len(dataSet))
            newEntropy +=prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        if (infoGain>bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

In [7]:
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] +=1
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [8]:
##### 얘가 제일 중요해!!!!!!!!!!!!!!!!!!!#####
def createTree(dataSet, labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet[0])==1:
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value]= createTree(splitDataSet (dataSet, bestFeat, value), subLabels)
    return myTree

In [24]:
myDat, labels = createDataSet()
myTree = createTree(myDat, labels)

In [20]:
print(myTree)

{'no surfing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}


In [11]:
def retrieveTree(i):
    listOfTrees = [{'no surfing':{0: 'no', 1:{'flippers':{0:'no',1:'yes'}}}},
                  {'no surfing':{0: 'no', 1:{'flippers':{0:{'head':{0:'no',1:'yes'}},1:'no'}}}}]
    return listOfTrees[i]

In [21]:
def classify(inputTree,featLabels,testVec):
    firstStr= list(inputTree)[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat=secondDict[key]
    if isinstance(valueOfFeat,dict):
        classLabel = classify(valueOfFeat, featLabels, testVec)
    else:
        classLabel = valueOfFeat
    return classLabel

In [32]:
print(classify(myTree, labels, [1,1]))
print(classify(myTree, labels, [1,0]))
print(classify(myTree, labels, [0,1]))
print(classify(myTree, labels, [0,0]))

yes
no
no
no


In [27]:
myDat, labels = createDataSet()

In [28]:
labels

['no surfing', 'flippers']

In [41]:
myTree['no surfing'][1]

{'flippers': {0: 'no', 1: 'yes'}}

<img src="img\DT.png">